# 2º Trabalho prático - Introdução a Banco de Dados
# Viagens a Serviço do Governo Federal (SCDP) 


#### Link Streamlit: (https://trabalho-pratico2-ibd.streamlit.app/)


# Membros:

#### Clarice Oliveira Minobolli Teixeira - 2023087761
#### João Carlos Ferraz de Sousa - 2023087990
#### Luiz Felipe Gondim Araujo - 2023028188
#### Matheus Costa Melo Silva - 2023087826


# Descrição dos dados

#### Este estudo foi desenvolvido com base nos dados públicos disponíveis no Sistema de Concessão de Diárias e Passagens (SCDP), acessados diretamente pelo portal de dados do governo federal (dados.gov.br). O SCDP é responsável por gerenciar os processos relacionados às viagens a serviço do Governo Federal, incluindo a concessão de diárias e passagens, garantindo maior controle físico e financeiro dessas operações no âmbito da Administração Pública Federal.

#### Para este trabalho, utilizamos especificamente os dados de bilhetes aéreos adquiridos no mês de junho de 2024. Este período foi selecionado por apresentar um volume significativo de informações, suficiente para realizar análises representativas e detalhadas, sem comprometer a eficiência do processamento ou a qualidade das conclusões obtidas.

#### Além disso, os dados foram enriquecidos com informações complementares sobre as aeronaves das principais companhias aéreas, como capacidade de assentos, status operacional e certificações emitidas pela ANAC. Essas informações adicionais foram extraídas de fontes públicas confiáveis, incluindo:

#### LATAM Airlines Group (https://www.aviacaocomercial.net/frotatam.htm);

#### Azul Linhas Aéreas (www.aviacaocomercial.net/frotaazul.htm);

#### GOL Linhas Aéreas (https://www.aviacaocomercial.net/frotagol.htm).

#### A integração desses dados visou oferecer uma visão mais abrangente e contextualizada sobre as operações aéreas realizadas, permitindo análises mais completas e precisas.



# Diagrama ER

![Diagrama ER](Modelo_ER.jpeg)

### Entidades presentes:                                                                               
#### Órgão 
#### Bilhete 
#### Preço
#### Penalidade
#### Companhia aérea 
#### Avião

### Relacionamentos presentes:
#### Superior - Hierarquia dos Órgãos
#### Orgao_bilhete - Órgão e Bilhete 
#### Bilhete_Penalidade - Bilhete e Penalidade
#### Bilhete_Preco - Bilhete e Preço
#### Bilhete_companhia - Bilhete e Companhia aérea
#### Companhia_aviao - Companhia aérea e avião

### Número de instâncias totais: 
#### 3954

# Diagrama Relacional

### Tabela: orgao
#### codigo_orgao (PK), nome_orgao, codigo_orgao_superior (FK)

### Tabela: orgao_bilhete
#### localizador (PK), codigo_orgao (PK)

### Tabela: bilhete
#### localizador (PK), data_emissao, situacao_bilhete, valor_bilhete, percentual_desconto_aplicado, valor_tarifa_comercial, valor_tarifa_governo, valor_tarifa_embarque, valor_multas, valor_reembolso, diferenca_tarifa, no_show, remarcado, cancelado, classe_tarifaria, data_embarque, numero_pcdp, id_companhia_aerea (FK)

### Tabela: companhia_aerea
#### id_companhia_aerea (PK), nome_companhia, regra_tarifaria

### Tabela: companhia_aerea
#### id_companhia_aerea (PK), id_aviao (PK), inativos, ativos

### Tabela: aviao
#### id_aviao (PK), modelo, assentos, selo_anac, motor

# Consultas

#### As consultas realizadas seguiram a proposta inicial do estudo, abordando aspectos fundamentais relacionados ao uso de bilhetes aéreos pelo governo. No entanto, com a integração de dados adicionais sobre as aeronaves de cada companhia aérea, expandimos o escopo para incluir consultas mais aprofundadas que explorassem essas informações.

#### Essas novas consultas permitiram analisar características específicas das aeronaves, como capacidade, status operacional e conformidade com regulamentações, enriquecendo as análises e proporcionando uma visão mais completa da base de dados. Essa abordagem complementou a proposta original, trazendo maior contextualização e relevância aos resultados apresentados.

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Bilhete mais caro

#### A consulta identifica o bilhete mais caro registrado na base de dados, considerando apenas bilhetes válidos (não cancelados e sem ocorrência de no-show). As informações retornadas incluem o localizador, o valor do bilhete e a data de embarque, ordenadas pelo valor do bilhete e em ordem decrescente.
#### Embora limitada, essa consulta fornece um dado relevante que pode servir como ponto de partida para investigações mais profundas, como:
#### Identificar o órgão responsável pela compra do bilhete mais caro.
#### Verificar se o custo está associado a um evento ou emergência específica.
#### Relacionar o custo elevado às características do voo (companhia, rota, tipo de aeronave).

In [4]:
query = """
            SELECT localizador, valor_bilhete, data_embarque
            FROM bilhete
            WHERE no_show = FALSE OR cancelado = FALSE
            ORDER BY valor_bilhete DESC
            LIMIT 1
        """

df = pd.read_sql_query(query, connection)
df

,localizador,valor_bilhete,data_embarque
0,FTWGKW,5847.14,30/06/2024


### 6.1.2 Média de preço dos bilhetes

#### A consulta tem como objetivo calcular a média do valor dos bilhetes aéreos adquiridos por órgãos registrados no banco de dados. Inicialmente, ela seleciona os códigos dos órgãos a partir da tabela orgao, em seguida, na tabela orgao_bilhete, são identificados os bilhetes comprados por esses órgãos. Por fim, a tabela bilhete é utilizada para calcular a média dos valores desses bilhetes. O resultado final é uma média consolidada dos preços dos bilhetes comprados pelos órgãos.
#### O resultado da consulta fornece uma visão global do custo médio dos bilhetes emitidos pelos órgãos. Esse valor é útil para avaliar a eficiência financeira na compra dos bilhetes e também pode ser um indicador de possíveis ajustes no orçamento destinado a viagens aéreas. A média pode ser comparada ao longo do tempo para identificar tendências, como um aumento ou diminuição nos custos.

In [5]:
query = """
            SELECT AVG(valor_bilhete) AS media_preco_bilhetes
            FROM bilhete
            WHERE localizador IN (
                SELECT localizador
                FROM orgao_bilhete
                WHERE codigo_orgao IN (
                    SELECT codigo_orgao
                    FROM orgao
                )
            )
        """

df = pd.read_sql_query(query, connection)
df

,media_preco_bilhetes
0,1529.925852


## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Companhias aereas que não possuem nenhum avião com o selo B da ANAC

#### Esta consulta identifica companhias aéreas que não possuem aviões com certificação "B" da ANAC. A junção das tabelas companhia_aerea_aviao e aviao permite verificar quais companhias possuem aeronaves com este selo, e a subconsulta exclui essas companhias do resultado final. É importante para assegurar que os contratos sejam firmados com empresas que atendam a padrões mínimos de segurança ou qualidade técnica, podendo embasar políticas de seleção mais criteriosas.

In [6]:
query = """
            SELECT DISTINCT id_companhia_aerea
            FROM companhia_aerea_aviao
            WHERE id_companhia_aerea NOT IN (
                SELECT id_companhia_aerea
                FROM companhia_aerea_aviao
                NATURAL JOIN aviao
                WHERE selo_anac = "B")
        """

df = pd.read_sql_query(query, connection)
df

,id_companhia_aerea
0,2


### 6.2.2 Órgãos que compraram mais de 30 bilhetes pro mesmo dia (embarque)

#### Esta consulta identifica órgãos que adquiriram mais de 30 bilhetes para embarque no mesmo dia. A junção entre as tabelas orgao, orgao_bilhete, e bilhete permite conectar os códigos dos órgãos aos bilhetes emitidos. Os resultados são agrupados pelo órgão e pela data de embarque, retornando o número total de bilhetes adquiridos por dia.

#### Relaciona-se à proposta ao fazer a análise de gastos por órgão, pois ajuda a identificar padrões específicos de alta demanda de bilhetes, o que pode refletir em ações conjuntas (como eventos ou deslocamentos em massa) e em possíveis impactos no orçamento do órgão. Além disso, é útil para detectar comportamentos operacionais únicos, como viagens coletivas, que podem indicar a necessidade de políticas específicas de planejamento ou contratação.

In [7]:
query = """
            SELECT 
                codigo_orgao,
                nome_orgao,
                data_embarque,
                COUNT(*) AS quantidade_bilhetes
            FROM 
                orgao
            NATURAL JOIN orgao_bilhete
            NATURAL JOIN bilhete
            GROUP BY 
                codigo_orgao, nome_orgao, data_embarque
            HAVING 
                COUNT(*) > 30
            ORDER BY 
                data_embarque, quantidade_bilhetes DESC;
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao,data_embarque,quantidade_bilhetes
0,101068,Conselho Administrativo de Recursos Fiscais,08/07/2024,32
1,223030,Secretaria Nacional dos Direitos da Pessoa com...,14/07/2024,428
2,223030,Secretaria Nacional dos Direitos da Pessoa com...,17/07/2024,326
3,223030,Secretaria Nacional dos Direitos da Pessoa com...,18/07/2024,104
4,221418,Departamento de Segurança Presidencial,20/06/2024,50
5,309260,Secretaria-Executiva do Conselho de Desenvolvi...,26/06/2024,49
6,309260,Secretaria-Executiva do Conselho de Desenvolvi...,27/06/2024,74
7,221418,Departamento de Segurança Presidencial,28/06/2024,44


### 6.2.3 Companhias aéreas com mais cancelamentos e/ou no shows

#### A consulta analisa o desempenho das companhias aéreas, calculando o percentual de bilhetes cancelados ou marcados como no-show. As tabelas companhia_aerea e bilhete são unidas para associar os bilhetes aos seus respectivos emissores. A métrica é obtida com base no número de bilhetes emitidos e no número de bilhetes cancelados ou com no-show.

#### É possivel identificar padrões de uso das companhias aéreas, ajudando a avaliar a eficiência e a confiabilidade das companhias contratadas. Esse dado pode ser usado para identificar padrões de desempenho e, potencialmente, para renegociar contratos com empresas menos eficientes.

In [8]:
query = """
            SELECT nome_companhia, COUNT(DISTINCT localizador) as nBilhetes, 
            COUNT(DISTINCT CASE WHEN no_show = TRUE OR cancelado = TRUE THEN localizador END) as nCancelamentos,
            ROUND(
                100.0 * COUNT(DISTINCT CASE WHEN no_show = TRUE OR cancelado = TRUE THEN localizador END) / COUNT(DISTINCT localizador), 2) AS pctCancelamentos
            FROM companhia_aerea
            NATURAL JOIN bilhete
            GROUP BY id_companhia_aerea
            ORDER BY pctCancelamentos DESC
        """

df = pd.read_sql_query(query, connection)
df

,nome_companhia,nBilhetes,nCancelamentos,pctCancelamentos
0,AZUL LINHAS AÉREAS BRASILEIRA,771,80,10.38
1,LATAM AIRLINES BRASIL,1938,143,7.38
2,GOL Linhas Aéreas Inteligentes,1245,69,5.54


## 6.3 Três consultas envolvendo junção de três ou mais relações

### 6.3.1 Órgão que comprou o bilhete mais caro gerado pela LATAM

#### Identifica o órgão que adquiriu o bilhete mais caro emitido pela LATAM. A junção das tabelas orgao, orgao_bilhete, bilhete, e companhia_aerea conecta os órgãos aos bilhetes emitidos e filtra os resultados apenas para bilhetes da LATAM. O resultado final é ordenado pelo valor do bilhete em ordem decrescente, retornando apenas o maior.

#### Esta consulta faz a análise de gastos por órgão, pois examina os custos elevados de bilhetes, possibilitando identificar órgãos que realizam compras de alto valor e destaca casos de maior custo por órgão, o que pode indicar necessidade de ajustes em práticas de compra ou renegociação com fornecedores específicos.

In [9]:
query = """
            SELECT DISTINCT codigo_orgao, nome_orgao, valor_bilhete
            FROM orgao
            NATURAL JOIN orgao_bilhete
            NATURAL JOIN bilhete
            NATURAL JOIN companhia_aerea
            WHERE nome_companhia == "LATAM AIRLINES BRASIL"
            ORDER BY
                valor_bilhete DESC
            LIMIT 1
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao,valor_bilhete
0,78055,Secretaria Especial de Articulação e Monitoram...,5847.14


### 6.3.2 Aviões ativos por modelo de avião e companhia

#### Lista os modelos de aviões ativos em operação por cada companhia aérea, com informações adicionais como o número de assentos disponíveis. A junção das tabelas companhia_aerea, companhia_aerea_aviao, e aviao permite cruzar informações sobre as companhias, seus modelos de aeronaves, e o status de operação.

#### A consulta é essencial para entender a infraestrutura e capacidade operacional de cada companhia aérea, permitindo avaliar se estão aptas a atender às demandas contratuais.

In [10]:
query = """
            SELECT nome_companhia, modelo, assentos, ativos
            FROM companhia_aerea
            NATURAL JOIN companhia_aerea_aviao 
            NATURAL JOIN aviao
            WHERE ativos > 0
        """

df = pd.read_sql_query(query, connection)
df

,nome_companhia,modelo,assentos,ativos
0,LATAM AIRLINES BRASIL,Airbus A319,144,19
1,LATAM AIRLINES BRASIL,Airbus A320,174,60
2,LATAM AIRLINES BRASIL,Airbus A320neo,174,16
3,LATAM AIRLINES BRASIL,Airbus A321,224,31
4,LATAM AIRLINES BRASIL,Airbus A321neo,228,14
5,LATAM AIRLINES BRASIL,Boeing 777-300ER,410,10
6,LATAM AIRLINES BRASIL,Boeing 787-9,303,1
7,GOL Linhas Aéreas Inteligentes,Boeing 737-700,138,11
8,GOL Linhas Aéreas Inteligentes,Boeing 737-800,186,56
9,GOL Linhas Aéreas Inteligentes,Boeing 737 MAX 8,186,52


### 6.3.3 Órgãos que mais cancelaram ou não apareceram em bilhetes comprados

#### A consulta determina quais órgãos superiores registraram o maior número de bilhetes cancelados ou com no-show. Através da junção de orgao, orgao_bilhete, e bilhete, conecta-se os bilhetes aos órgãos subordinados e superiores. Apenas órgãos superiores (sem vínculo hierárquico acima) são considerados no agrupamento e no somatório de cancelamentos.
####  Remete a análise da prosposta, Eficiência Operacional das Viagens, já que analisa o impacto de cancelamentos e no-shows, indicadores de baixa eficiência administrativa e possíveis desperdícios.Além disso, permite identificar órgãos superiores com alta taxa de ineficiência nas viagens, auxiliando no planejamento e na tomada de decisões para reduzir custos e melhorar processos.

In [11]:
query = """
            SELECT nome_superior, SUM(nViagens) as cancelamentos
            FROM (
                SELECT o2.nome_orgao as nome_superior, COUNT(DISTINCT b.localizador) AS nViagens
                FROM orgao AS o1
                JOIN orgao AS o2 ON o1.codigo_orgao_superior = o2.codigo_orgao
                JOIN orgao_bilhete AS ob ON o1.codigo_orgao = ob.codigo_orgao
                JOIN bilhete AS b ON ob.localizador = b.localizador
                WHERE o2.codigo_orgao_superior IS NULL AND (b.no_show = TRUE OR b.cancelado = TRUE)
                GROUP BY o1.codigo_orgao
            ) AS subquery
            GROUP BY nome_superior
            ORDER BY cancelamentos DESC
        """

df = pd.read_sql_query(query, connection)
df

,nome_superior,cancelamentos
0,Advocacia-Geral da União,48
1,Gabinete de Segurança Institucional da Presidê...,38
2,Ministério dos Direitos Humanos e da Cidadania,37
3,Secretaria de Relações Institucionais,30
4,Ministério da Gestão e da Inovação em Serviços...,24
5,Ministério das Cidades,19
6,Secretaria-Geral da Presidência da República,18
7,Ministério da Integração e do Desenvolvimento ...,16
8,Ministério da Fazenda,14
9,Casa Civil da Presidência da República,14


## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Orgão cuja soma das multas com bilhetes aéreos é maior

#### A consulta identifica o órgão com maior soma de multas associadas aos bilhetes aéreos. A junção entre as tabelas orgao, orgao_bilhete, e bilhete conecta os bilhetes às multas registradas e aplica uma agregação para calcular a soma total de multas por órgão. O resultado é ordenado de forma decrescente, retornando o órgão com maior soma.

#### Este levantamento ajuda a identificar quais órgãos acumulam maiores custos com multas, destacando a necessidade de melhorar a gestão de viagens e os processos de compra de bilhetes, a fim de reduzir essas penalidades.

In [12]:
query = """
            SELECT DISTINCT codigo_orgao, nome_orgao, SUM(valor_multas) as soma_das_multas
            FROM orgao
            NATURAL JOIN orgao_bilhete
            NATURAL JOIN bilhete
            WHERE valor_multas > 0 
            GROUP BY codigo_orgao, nome_orgao
            ORDER BY soma_das_multas desc
            LIMIT 1
            
        """

df = pd.read_sql_query(query, connection)
df

,codigo_orgao,nome_orgao,soma_das_multas
0,70352,Secretaria-Geral de Administração,9562.89


### 6.4.2  Número de viagens de cada órgão superior

#### Esta consulta calcula o número total de viagens realizadas por cada órgão superior, somando os bilhetes válidos (não cancelados e sem no-show). A consulta utiliza junção entre orgao, orgao_bilhete, e bilhete, agrupando os dados por órgão superior para realizar a agregação da soma do número de viagens.

#### A análise do número de viagens permite identificar quais órgãos superiores concentram maior atividade operacional, possibilitando ajustes no planejamento logístico e no orçamento de viagens.

In [13]:
query = """
            SELECT nome_superior, SUM(nViagens) as tViagens
            FROM (
                SELECT o2.nome_orgao as nome_superior, COUNT(DISTINCT b.localizador) AS nViagens
                FROM orgao AS o1
                JOIN orgao AS o2 ON o1.codigo_orgao_superior = o2.codigo_orgao
                JOIN orgao_bilhete AS ob ON o1.codigo_orgao = ob.codigo_orgao
                JOIN bilhete AS b ON ob.localizador = b.localizador
                WHERE o2.codigo_orgao_superior IS NULL AND b.no_show = FALSE AND b.cancelado = FALSE
                GROUP BY o1.codigo_orgao
            ) AS subquery
            GROUP BY nome_superior
            ORDER BY tViagens DESC
        """

df = pd.read_sql_query(query, connection)
df

,nome_superior,tViagens
0,Ministério dos Direitos Humanos e da Cidadania,1264
1,Gabinete de Segurança Institucional da Presidê...,380
2,Secretaria de Relações Institucionais,258
3,Ministério das Cidades,238
4,Ministério da Fazenda,218
5,Advocacia-Geral da União,207
6,Ministério da Integração e do Desenvolvimento ...,197
7,Casa Civil da Presidência da República,173
8,Secretaria-Geral da Presidência da República,150
9,Ministério da Gestão e da Inovação em Serviços...,141


# Autoavalição dos membros

#### "Durante o trabalho em grupo, contribuí ativamente no processo de normalização da base de dados, garantindo que as tabelas fossem estruturadas de forma consistente e eficiente. Além disso, desenvolvi os scripts SQL para criar as tabelas no banco de dados e populá-las com os dados normalizados, e, por fim, ainda implementei a interface em Streamlit. Considero que minha contribuição foi fundamental para a realização do trabalho, haja vista que participei da maioria das etapas de desenvolvimento." - Luiz Felipe Araújo Gondim

#### "Para o trabalho em grupo, me responsabilizei pela elaboração de 5 consultas SQL, contribui para o processo de normalização da base de dados, sempre buscando puxar debates e a atenção do grupo para a consistência do esquema relacional normalizado. Considero que minha participação foi positiva para o grupo, sempre tentando resolver minhas pendências com antecedência e fulgor." - João Carlos Ferraz de Souza

#### "Nesse trabalho, participei ativamente ajudando na criação do esquema ER, relacional e na normalização dessas tabelas. Além disso, formulei 5 das 10 consultas SQL ao banco de dados, buscando sempre realizar consultas completas e a aplicabilidade das mesmas no mundo real, o que nos permitirá fazer uma análise mais contundente." - Clarice Oliveira Minobolli Teixeira

#### " Durante o trabalho em grupo, fui responsável pela elaboração da proposta inicial e pela análise final das consultas realizadas ao banco de dados. Conduzi a análise dos resultados, associando-os aos objetivos da proposta, como análise de gastos, eficiência operacional e padrões de uso das companhias aéreas. Acredito que minha contribuição foi essencial para o sucesso do trabalho, pois assegurei que as consultas e suas análises estivessem alinhadas com os objetivos do projeto, proporcionando uma visão clara e objetiva dos dados." - Matheus Costa Melo Silva